# File to process data from survey among acquaintances
## from importing data to processing and visualizing

In [1]:
# Importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

## Importing data

In [2]:
# Importing data
df_whatsapp_data = pd.read_csv('data/umfrage-nachhaltige-mobilitaet_bekannte.csv')

## Methods for data processing

In [ ]:
def explore_data(data):
    #Display a summary of the data
    # args: data: pandas dataframe
    # Summarize the data
    print(data.info())
    # Display the first 5 rows of the data
    print(data.head())
    
def drop_data(data):
    # Drop non-uniques and missing values
    # args: data: pandas dataframe
    # Drop duplicates
    duplicate_free_data = data.drop_duplicates()
    #data.drop_duplicates(inplace=True)
    # Drop missing values
    na_free_data = duplicate_free_data.dropna(inplace=True)
    # Replace missing values with the mean
    #data.fillna(data.mean(), inplace=True)
    return na_free_data

def fill_data(data):
    # Fill missing values with the mean
    # args: data: pandas dataframe
    #data.fillna(data.mean(), inplace=True)
    data.fillna(method='ffill').fillna(method='bfill')
    return data

def split_data(data):
    # split data for use case
    # args: data: pandas dataframe
    # Split the data into features and target
    data_without_na = data.dropna()
    selected_columns_finace = ['financial_situation', 'financial_situation_future', 'financial_situation_past']#TODO select proper columns for correlation with financial situation
    selected_columns_mobility = ['mobility', 'mobility_future', 'mobility_past']#TODO select proper columns for correlation with mobility
    selected_columns_other = ['age',''] #TODO select proper columns for correlation with 'other' data
    data_financial_correlation = data[selected_columns_finace]
    data_mobility_correlation = data[selected_columns_mobility]
    data_other_correlation = data[selected_columns_other]
    return data, data_without_na, data_financial_correlation, data_mobility_correlation, data_other_correlation

def prepare_data(data, feature_columns, target_column):
    # Prepare the data for analysis
    # args: data: pandas dataframe
    # args: feature_column: (list): liste der spalten die als feature genutzt werden sollen
    # args: target_column: (string): name zielvariable
    #return: tuple: X_train, X_test, y_train, y_test
    X = data[feature_columns]
    Y = data[target_column]
    # Split the data into training and testing data
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
    return X_train, X_test, y_train, y_test

def scale_data(X_train, X_test):
    # Scale the data
    # args: X_train: pandas dataframe
    # args: X_test: pandas dataframe
    #return: X_train_scaled, X_test_scaled
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    return X_train_scaled, X_test_scaled

def train_model(X_train, y_train):
    # Train the model
    # args: X_train: pandas dataframe
    # args: y_train: pandas dataframe
    #return: model
    model = LinearRegression()
    model.fit(X_train, y_train)
    return model

def evaluate_model(model, X_test, y_test):
    # Evaluate the model
    # args: model: model
    # args: X_test: pandas dataframe
    # args: y_test: pandas dataframe
    #return: mse
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    return mse

def analyze_data(X_train, X_test, Y_train, Y_test):
    # Analyze the data
    # args: X_train: pandas dataframe
    # args: X_test: pandas dataframe
    # args: Y_train: pandas dataframe
    # args: Y_test: pandas dataframe
    #return: mse
    # X_train_scaled, X_test_scaled = scale_data(X_train, X_test)
    # model = train_model(X_train_scaled, Y_train)
    # mse = evaluate_model(model, X_test_scaled, Y_test)
    # return mse
    model = LinearRegression()
    model.fit(X_train, Y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(Y_test, y_pred)
    print('Mean Squared Error:', mse)
    return mse

def visualize_data(data):
    # TODO Visualize the data with matplotlib and seaborn
    # args: data: pandas dataframe
    #return: None
    # TODO Implement the visualization
    print('Visualizing the data')
    

## Data Exploration

In [6]:
# Displaying the first 20 rows of the data
df_whatsapp_data.head(20)
# count of columns
#df_whatsapp_data.count()

,Zeitstempel,Wie alt sind Sie?,Wie ist Ihre Wohnsituation? (Wohneigentum?),Wie ist Ihre Wohnsituation? (Haus oder Wohnung?),"Wie ist Ihre Wohnsituation? (Großstadt, Stadt, Land?)",Wie ist Ihre Beschäftigung?,"Bitte schätzen Sie das monatliche Netto Gesamteinkommen in Ihrem Haushalt in Euro. (Wenn Sie dazu keine Angaben machen möchten, überspringen Sie diese Frage bitte.)",Besitzen Sie ein Auto? (auch Leasing),"Wie interessiert sind Sie an einem E-Auto als Ihr nächstes(/erstes) KFZ? (Wenn Sie generell kein Interesse am Besitz eines Autos haben, überspringen Sie diese Frage bitte)","Welchen Anteil an Ihrer Gesamtmobilität macht Ihr Auto aus? (falls Sie kein Auto besitzen, wählen Sie bitte 0% aus)",Welchen Anteil an Ihrer Gesamtmobilität macht der ÖPNV aus?,Was würde den ÖPNV für Sie attraktiver machen?,Was würde E-Autos für Sie attraktiver machen?
0,2024/04/29 4:32:21 PM OESZ,46-55,gemietet,Wohnung,Stadt,Angestellter,NaN,Auto,10,50%,0%,Besseres Netz,günstigere Preise;mehr/besser verteilte Lademö...
1,2024/04/29 4:38:37 PM OESZ,36-45,Wohneigentum,Haus,Land,Angestellter,NaN,Auto,6,90%,0%,günstigere Preise;kürzere Wartezeit zwischen z...,günstigere Preise;mehr/besser verteilte Lademö...
2,2024/04/29 4:39:08 PM OESZ,26-35,gemietet,Reihenhaus,Land,Angestellter,NaN,E-Auto,6,80%,0%,günstigere Preise,"mehr Reichweite;mehr Sicherheit (Brandgefahr, ..."
3,2024/04/29 4:39:18 PM OESZ,26-35,Wohneigentum,Haus,Land,Angestellter,7000,E-Auto,7,80%,20%,günstigere Preise;kürzere Wartezeit zwischen z...,günstigere Preise;gute Budget-E-Autos;mehr/bes...
4,2024/04/29 4:39:39 PM OESZ,36-45,gemietet,Haus,Land,Angestellter,4600,Auto,7,90%,0%,kürzere Wartezeit zwischen zwei Zügen/Bussen;b...,"mehr Reichweite;mehr Sicherheit (Brandgefahr, ..."
5,2024/04/29 4:41:40 PM OESZ,26-35,gemietet,Wohnung,Großstadt (min. 100.000 Einwohner),Angestellter,NaN,Auto,8,80%,10%,günstigere Preise,günstigere Preise;mehr/besser verteilte Lademö...
6,2024/04/29 4:44:42 PM OESZ,26-35,gemietet,Wohnung,Land,Angestellter,4200€,Auto,9,80%,20%,günstigere Preise;kürzere Wartezeit zwischen z...,mehr/besser verteilte Lademöglichkeiten;mehr R...
7,2024/04/29 4:45:26 PM OESZ,56-65,gemietet,Wohnung,Stadt,Angestellter,NaN,Auto,1,100%,0%,Keine Möglichkeit der Nutzung,mehr Reichweite
8,2024/04/29 4:45:28 PM OESZ,36-45,gemietet,Wohnung,Großstadt (min. 100.000 Einwohner),Angestellter,2700,Auto,7,40%,10%,günstigere Preise;Bessere Anbindung außerhalb ...,günstigere Preise;gute Budget-E-Autos;mehr/bes...
9,2024/04/29 4:49:05 PM OESZ,36-45,gemietet,Wohnung,Stadt,Angestellter,NaN,Auto,6,80%,10%,günstigere Preise;geringere Distanz zwischen H...,mehr/besser verteilte Lademöglichkeiten;mehr R...


## Data Cleaning

In [ ]:
# Checking for missing values
df_whatsapp_data.isnull().sum()

In [ ]:
# Dropping the missing values
df_whatsapp_data.dropna(inplace=True)

In [ ]:
# Checking for duplicates
df_whatsapp_data.duplicated().sum()

In [ ]:
# Dropping duplicates
df_whatsapp_data.drop_duplicates(inplace=True)

In [ ]:
# Checking for unique values in the data
df_whatsapp_data.nunique()

In [ ]:
# Replacing the missing values with the average values and fitting strings
df_whatsapp_data.fillna(df_whatsapp_data.mean(), inplace=True)

## Data Preprocessing

In [ ]:
# Checking the data types of the columns
df_whatsapp_data.dtypes

## Data Analysis

In [ ]:
# Checking the distribution of the data
df_whatsapp_data.describe()

## Data Visualization

In [ ]:
# Plotting the distribution of the data
df_whatsapp_data.hist(figsize=(20, 20))
plt.show()
# add seaborn style
sns.set()
# add title and labels
plt.title('Distribution of the data')
plt.xlabel('Values')
plt.ylabel('Frequency')
plt.show()